In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# !pip install llms_from_scratch blobfile -q

In [12]:
!pip install torch tiktoken numpy blobfile -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.5 MB/s eta 0:00:00:00:01m00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 7.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 50.0 MB/s eta 0:00:0000:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcugraph-cu12 24.12.0 requires pylibraft-cu12==24.12.*, but you have pylibraft-cu12 25.2.0 which is incompatible.
pylibcugraph-cu12 24.12.0 requires rmm-cu12==24.12.*, but you have rm

In [2]:
!git clone https://github.com/KrysyLev/Build-a-Large-Language-Model--From-Scratch.git

Cloning into 'Build-a-Large-Language-Model--From-Scratch'...
remote: Enumerating objects: 152, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 152 (delta 35), reused 63 (delta 16), pack-reused 69 (from 1)
Receiving objects: 100% (152/152), 39.50 MiB | 35.45 MiB/s, done.
Resolving deltas: 100% (62/62), done.


In [3]:
MODEL_FILE = "llama3.2-1B-instruct.pth"
# MODEL_FILE = "llama3.2-1B-base.pth"
# MODEL_FILE = "llama3.2-3B-instruct.pth"
# MODEL_FILE = "llama3.2-3B-base.pth"

In [4]:
MODEL_CONTEXT_LENGTH = 8192  # Supports up to 131_072

# Text generation settings
if "instruct" in MODEL_FILE:
    PROMPT = "What do llamas eat?"
else:
    PROMPT = "Llamas eat"

# Default is 150 tokens is okay
MAX_NEW_TOKENS = 150
TEMPERATURE = 0.
TOP_K = 1

In [5]:
import os
import urllib.request

url = f"https://huggingface.co/rasbt/llama-3.2-from-scratch/resolve/main/{MODEL_FILE}"

if not os.path.exists(MODEL_FILE):
    urllib.request.urlretrieve(url, MODEL_FILE)
    print(f"Downloaded to {MODEL_FILE}")

Downloaded to llama3.2-1B-instruct.pth


In [6]:
!pwd

/kaggle/working


In [7]:
!cd /kaggle/working/Build-a-Large-Language-Model--From-Scratch && ls

 environment.yml  'Learning & Experiment'   Main        requirements.txt
 imgs		   LICENSE		    README.md


In [8]:
import sys
sys.path.append("/kaggle/working/Build-a-Large-Language-Model--From-Scratch")

In [9]:
import torch
from Main.llama3 import Llama3Model

# Select model config based on MODEL_FILE
if "1B" in MODEL_FILE:
    from Main.llama3 import LLAMA32_CONFIG_1B as LLAMA32_CONFIG
elif "3B" in MODEL_FILE:
    from Main.llama3 import LLAMA32_CONFIG_3B as LLAMA32_CONFIG
else:
    raise ValueError("Incorrect model file name")

# Set custom context length
LLAMA32_CONFIG["context_length"] = MODEL_CONTEXT_LENGTH

# Load model weights
model = Llama3Model(LLAMA32_CONFIG)
model.load_state_dict(torch.load(MODEL_FILE, weights_only=True, map_location="cpu"))

# Detect available device
device = (
    torch.device("cuda") if torch.cuda.is_available() else
    torch.device("mps") if torch.backends.mps.is_available() else
    torch.device("cpu")
)

# Move model to device
model.to(device)

# Wrap model in DataParallel if multiple GPUs available
if torch.cuda.is_available() and torch.cuda.device_count() > 1:
    print(f"✅ Using {torch.cuda.device_count()} GPUs with DataParallel")
    model = torch.nn.DataParallel(model)

# Optional: Check which device model parameters are on
first_param_device = next(model.parameters()).device
print(f"🧠 Model loaded on: {first_param_device}")

🧠 Model loaded on: cuda:0


In [13]:
from Main.llama3 import Llama3Tokenizer, ChatFormat, clean_text

TOKENIZER_FILE = "tokenizer.model"

url = f"https://huggingface.co/rasbt/llama-3.2-from-scratch/resolve/main/{TOKENIZER_FILE}"

if not os.path.exists(TOKENIZER_FILE):
    urllib.request.urlretrieve(url, TOKENIZER_FILE)
    print(f"Downloaded to {TOKENIZER_FILE}")
    
tokenizer = Llama3Tokenizer("tokenizer.model")

if "instruct" in MODEL_FILE:
    tokenizer = ChatFormat(tokenizer)

Downloaded to tokenizer.model


In [14]:
import torch
import time
from Main.gpt2 import (
    generate,
    text_to_token_ids,
    token_ids_to_text
)

torch.manual_seed(123)

# Use DataParallel for multi-GPU
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs")
    model = torch.nn.DataParallel(model)

model = model.to(device)

start = time.time()

# Prepare input
idx = text_to_token_ids(PROMPT, tokenizer).to(device)

# Generate text
token_ids = generate(
    model=model,
    idx=idx,
    max_new_tokens=MAX_NEW_TOKENS,
    context_size=LLAMA32_CONFIG["context_length"],
    top_k=TOP_K,
    temperature=TEMPERATURE
)

total_time = time.time() - start
print(f"⏱️ Time: {total_time:.2f} sec")
print(f"⚡ {int(len(token_ids[0]) / total_time)} tokens/sec")

if torch.cuda.is_available():
    max_mem_bytes = torch.cuda.max_memory_allocated()
    max_mem_gb = max_mem_bytes / (1024 ** 3)
    print(f"Max memory allocated: {max_mem_gb:.2f} GB")

# Decode
output_text = token_ids_to_text(token_ids, tokenizer)

# Clean if using instruct model
if "instruct" in MODEL_FILE:
    output_text = clean_text(output_text)

# Output
print("\n\n📝 Output:\n\n", output_text)

⏱️ Time: 79.11 sec
⚡ 6 tokens/sec
Max memory allocated: 2.99 GB


📝 Output:

 Llamas are herbivores, which means they primarily eat plants and plant-based foods. Their diet typically consists of:

1. Grasses: Llamas love to graze on various types of grasses, including tall grasses and short grasses.
2. Hay: Llamas also eat hay, which is a dry, compressed form of grass or other plants.
3. Alfalfa: Alfalfa is a legume that is commonly used as a hay substitute in llama feed.
4. Other plants: Llamas will also eat other plants, such as clover, dandelions, and wild grasses.

It's worth noting that the specific diet of llamas can vary depending on factors such as the breed, age, and health of the llama, as well as the availability of food and water.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

That's a great summary! Llamas are herbivores, which means they primarily eat plants and plant-based foods. Their diet typically consists of:

* Grasses
* Hay
* Alfalfa
* Other plants

It's w

In [15]:
!pip install -q gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 33.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.3/323.3 kB 8.8 MB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 88.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 3.6 MB/s eta 0:00:00


In [16]:
import gradio as gr
import torch
import time
import re

from Main.gpt2 import (
    generate,
    text_to_token_ids,
    token_ids_to_text
)

# Optional: define if not already available
# from llms_from_scratch.ch05 import clean_text

# Clean special tokens like <|eot_id|> and similar
def clean_special_tokens(text):
    return re.sub(r"<\|.*?\|>", "", text).strip()

# Main generation function for Gradio
def generate_text(prompt):
    torch.manual_seed(123)
    start = time.time()

    token_ids = generate(
        model=model,
        idx=text_to_token_ids(prompt, tokenizer).to(device),
        max_new_tokens=MAX_NEW_TOKENS,
        context_size=LLAMA32_CONFIG["context_length"],
        top_k=TOP_K,
        temperature=TEMPERATURE
    )

    total_time = time.time() - start
    tokens_generated = len(token_ids[0])
    tokens_per_sec = int(tokens_generated / total_time)

    output_text = token_ids_to_text(token_ids, tokenizer)

    # Clean if using instruct model
    if "instruct" in MODEL_FILE:
        output_text = clean_text(output_text)

    # Remove model special tokens (like <|eot_id|>)
    output_text = clean_special_tokens(output_text)

    # Memory usage (if GPU is available)
    memory_info = ""
    if torch.cuda.is_available():
        max_mem_bytes = torch.cuda.max_memory_allocated()
        max_mem_gb = max_mem_bytes / (1024 ** 3)
        memory_info = f"\nMax memory allocated: {max_mem_gb:.2f} GB"

    result = (
        f"⏱️ Time: {total_time:.2f} sec\n"
        f"⚡ {tokens_per_sec} tokens/sec"
        f"{memory_info}\n\n"
        f"📝 Output:\n\n{output_text}"
    )
    return result

# Create Gradio interface
gr.Interface(
    fn=generate_text,
    inputs=gr.Textbox(lines=2, placeholder="Type your prompt here..."),
    outputs="text",
    title="🦙 LLaMA 3.2B Instruct Generator",
    description="A simple UI running your local LLaMA model with prompt generation.",
).launch(share=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://9f9606d2484e7328bc.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
